**Poin penilaian :**

* Tingkat kesesuaian program
* Kesesuaian step preprocessing dan pemodelan
* Pemahaman materi
* Kerapian kode program
* Tidak ada error

# Content Based Filtering : Movie Recommender System

**Import library**

In [1]:
import pandas as pd
from math import sqrt
from zipfile import ZipFile
import numpy as np
import wget
import matplotlib.pyplot as plt
%matplotlib inline

**Gunakan dataset berikut**

In [2]:
# wget.download('http://files.grouplens.org/datasets/movielens/ml-latest.zip')
print('unziping ...')
# zf = ZipFile('ml-latest.zip', 'r')
# zf.extractall()
# zf.close()

unziping ...


Sekarang mari kita baca setiap file ke dalam Dataframe:

In [3]:
movies_df = pd.read_csv ('ml-latest/movies.csv')
ratings_df = pd.read_csv ('ml-latest/ratings.csv')
movies_df.head ()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


##### **Kegiatan 1 : 5 poin**
Hapus tahun dari kolom `title` dengan menggunakan fungsi `replace` pandas dan menyimpannya di kolom `year` yang baru.

In [4]:
# TULIS KODE ANDA DISINI
movies_df['year'] = movies_df['title'].str[-5:-1]
movies_df.title = movies_df.title.replace({" \\([0-9]+\\)":""}, regex=True)
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


##### **Kegiatan 2 : 5 poin**

Ubah kolom **Genres** menjadi sebuah **list of Genre**.

In [5]:
# TULIS KODE ANDA DISINI
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


##### **Kegiatan 3 : 15 poin**

Ubah kolom genres kedalam bentuk **One Hot Encoding** kemudian simpan dalam dataframe baru.

Karena genre dalam format list tidak optimal untuk teknik sistem rekomendasi berbasis konten, kita akan menggunakan teknik **One Hot Encoding** untuk mengubah daftar genre menjadi vektor di mana setiap kolom sesuai dengan satu kemungkinan nilai fitur. Pengkodean ini diperlukan untuk memasukkan data kategori. Dalam hal ini, kita akan menyimpan setiap genre berbeda dalam kolom yang berisi 1 atau 0. 1 menunjukkan bahwa film memiliki genre tersebut dan 0 menunjukkan bahwa itu tidak.

In [6]:
# TULIS KODE ANDA DISINI
moviesWithGenres_df = movies_df.copy()
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1

moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Selanjutnya, mari kita lihat pada dataframe ratings.

In [7]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


Setiap baris dalam dataframe ratings memiliki id pengguna yang terkait dengan setidaknya satu film, peringkat, dan timestamp yang ditampilkan ketika mereka memeriksanya. Kita tidak akan memerlukan kolom timestamp, jadi mari letakkan untuk menghemat memori.

In [8]:
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


<a id="ref3"></a>
## **Content-Based recommendation system**

Sekarang, mari kita lihat bagaimana menerapkan __Content-Based__ atau __Item-item Recommendation systems__. Teknik ini mencoba untuk mencari tahu apa aspek favorit pengguna dari suatu item dan kemudian merekomendasikan item yang menyajikan aspek-aspek tersebut. Dalam kasus kami, kami akan mencoba mencari tahu genre input favorit dari film dan peringkat yang diberikan.

Mari kita mulai dengan membuat pengguna input untuk merekomendasikan film ke:

Perhatikan: Untuk menambahkan lebih banyak film, cukup tambahkan jumlah elemen dalam __userInput__. Jangan ragu untuk menambahkan lebih banyak! Pastikan untuk menuliskannya dengan huruf kapital dan jika film dimulai dengan "The", seperti "The Matrix" kemudian tulis seperti ini: 'Matrix, The'.

##### **Kegiatan 4 : 15 poin**

Buat user input berupa list of dictionary yang terdiri dari 6 dictionary yang berisi judul film dan rating yang berbeda.

Ubah list of dictionary tersebut kedalam dataframe `inputMovies` dan tampilkan isi dataframe tersebut.

# **NOTE : Pastikan Tidak Ada Isi List of Dictionary Yang Sama Persis Dengan Praktikan Lain**

In [9]:
# TULIS KODE ANDA DISINI


userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Great Glinka, The', 'rating':4},
            {'title':'Jumanji', 'rating':2},
            {'title':"Flora", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,"Great Glinka, The",4.0
3,Jumanji,2.0
4,Flora,5.0
5,Akira,4.5


### **Add movieId to input user**

Dengan input yang lengkap, mari kita ekstrak ID film input dari bingkai data film dan menambahkannya ke dalamnya.

Kita dapat mencapai ini dengan terlebih dahulu memfilter baris yang berisi judul film input dan kemudian menggabungkan subset ini dengan input dataframe. Kami juga menjatuhkan kolom yang tidak perlu untuk input untuk menghemat ruang memori.

In [10]:
mv_id = movies_df[['movieId','title']]
inputMovies = pd.merge(mv_id, inputMovies, on='title')
inputMovies

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,1274,Akira,4.5
3,164600,Akira,4.5
4,1968,"Breakfast Club, The",5.0
5,193882,Flora,5.0


Kita akan mulai dengan mempelajari preferensi input, jadi mari kita ambil subset film yang inputnya tonton dari Dataframe yang mengandung genre yang didefinisikan dengan nilai-nilai biner.

In [11]:
#Menyaring film dari input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
44993,164600,Akira,"[Action, Crime, Thriller]",2016,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
58096,193882,Flora,"[Adventure, Drama, Horror, Sci-Fi]",2017,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Kita hanya perlu tabel genre aktual, jadi mari kita bersihkan ini sedikit.

##### **Kegiatan 5 : 10 Poin**
Atur ulang indeks drop kolom film Id, judul, genre dan kolom tahun.

In [12]:
# TULIS KODE ANDA DISINI
userMovies = userMovies.reset_index(drop=True)
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Sekarang kita mulai mempelajari preferensi input!

Untuk melakukan ini, kita akan mengubah setiap genre menjadi bobot. Kita dapat melakukan ini dengan menggunakan input ulasan dan mengalikannya ke dalam tabel genre input dan kemudian merangkum tabel yang dihasilkan oleh kolom. Operasi ini sebenarnya merupakan produk titik antara matriks dan vektor, jadi kita bisa melakukannya dengan memanggil fungsi "dot" Pandas.

In [13]:
inputMovies['rating']

0    3.5
1    2.0
2    4.5
3    4.5
4    5.0
5    5.0
Name: rating, dtype: float64

In [14]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Adventure             15.0
Animation              8.0
Children               5.5
Comedy                 8.0
Fantasy                5.5
Romance                0.0
Drama                  9.5
Action                 9.5
Crime                  5.0
Thriller               5.0
Horror                 5.0
Mystery                0.0
Sci-Fi                 9.5
IMAX                   0.0
Documentary            0.0
War                    0.0
Musical                0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Sekarang, kita memiliki bobot untuk setiap preferensi pengguna. Ini dikenal sebagai Profil Pengguna. Dengan menggunakan ini, kami dapat merekomendasikan film yang memenuhi preferensi pengguna.

Mari kita mulai dengan mengekstraksi tabel genre dari kerangka data asli:

In [15]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
genreTable.shape

(58098, 20)

##### **Kegiatan 6 : 30 Poin**

Dengan profil input dan daftar lengkap film serta genre mereka, Kita akan mengambil rata-rata tertimbang dari setiap film berdasarkan profil input dan merekomendasikan dua puluh film teratas yang paling memuaskan.

In [17]:
# LENGKAPI 2 KODE DIBAWAH
"""1. Multiply the genres by the weights and then take the weighted average"""
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.491228
2    0.304094
3    0.093567
4    0.204678
5    0.093567
dtype: float64

In [18]:
"""2. Sort our recommendations in descending order"""
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
# Tampilkan isi dataset
recommendationTable_df.head()

movieId
5018      0.672515
161594    0.666667
81132     0.666667
85261     0.649123
62956     0.649123
dtype: float64

## **Sekarang inilah tabel rekomendasi!**

##### **Kegiatan 7 : 20 Poin**

Buat tabel rekomendasi akhir.

In [19]:
#The final recommendation table

movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
542,546,Super Mario Bros.,"[Action, Adventure, Children, Comedy, Fantasy,...",1993
4923,5018,Motorama,"[Adventure, Comedy, Crime, Drama, Fantasy, Mys...",1991
9222,27155,"Batman/Superman Movie, The","[Action, Adventure, Animation, Children, Fanta...",1998
9300,27344,Revolutionary Girl Utena: Adolescence of Utena...,"[Action, Adventure, Animation, Comedy, Drama, ...",1999
10585,40339,Chicken Little,"[Action, Adventure, Animation, Children, Comed...",2005
11511,49593,She,"[Action, Adventure, Drama, Fantasy, Horror, Ro...",1965
11800,52287,Meet the Robinsons,"[Action, Adventure, Animation, Children, Comed...",2007
11821,52462,Aqua Teen Hunger Force Colon Movie Film for Th...,"[Action, Adventure, Animation, Comedy, Fantasy...",2007
12037,54278,Underdog,"[Action, Adventure, Children, Comedy, Fantasy,...",2007
13130,62956,Futurama: Bender's Game,"[Action, Adventure, Animation, Comedy, Fantasy...",2008
